In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn import linear_model

In [2]:
nasdaq_100_tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST", "PDD",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "ZM", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "DOCU", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "ASAN", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "FOXA", "FOX", "ANSS",
    "SWKS", "OKTA", "TTD", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "PTON", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]

In [3]:
training_data=pd.read_csv('training_full_data.csv', header=0)
training_data=training_data.set_index(['ticker', 'Date'])

In [4]:
training_data

Open       High        Low      Close  Adj Close  \
ticker Date                                                                
AAPL   2016/01/05  26.437500  26.462500  25.602501  25.677500  23.376617   
       2016/01/06  25.139999  25.592501  24.967501  25.174999  22.919142   
       2016/01/07  24.670000  25.032499  24.107500  24.112499  21.951851   
       2016/01/08  24.637501  24.777500  24.190001  24.240000  22.067924   
       2016/01/11  24.742500  24.764999  24.334999  24.632500  22.425255   
...                      ...        ...        ...        ...        ...   
WDC    2020/12/23  49.950001  50.599998  49.410000  49.930000  49.930000   
       2020/12/24  50.310001  50.380001  49.349998  49.919998  49.919998   
       2020/12/28  50.590000  50.970001  49.630001  50.060001  50.060001   
       2020/12/29  50.639999  50.639999  48.230000  48.290001  48.290001   
       2020/12/30  48.279999  50.290001  48.279999  49.529999  49.529999   

                      Volume   returns   ret_fwd  
ticker Date                                       
AAPL   2016/01/05  223164000 -0.660000 -0.025059  
       2016/01/06  273829600 -0.502501 -0.019570  
       2016/01/07  324377600 -1.062500 -0.042205  
       2016/01/08  283192000  0.127501  0.005288  
       2016/01/11  198957600  0.392500  0.016192  
...                      ...       ...       ...  
WDC    2020/12/23    2685500  0.209999  0.004224  
       2020/12/24    1224500 -0.010002 -0.000200  
       2020/12/28    2763400  0.140003  0.002805  
       2020/12/29    4622400 -1.770000 -0.035358  
       2020/12/30    3965400  1.239998  0.025678  

[119409 rows x 8 columns]

In [5]:
training_adj_close=pd.DataFrame()
training_log_ret=pd.DataFrame()
for ticker in nasdaq_100_tickers:
    training_adj_close[ticker]=training_data.loc[ticker]['Adj Close']
    daily_log_return = np.log(training_adj_close[ticker] / training_adj_close[ticker].shift(1))
    training_log_ret[ticker] = daily_log_return
training_adj_close

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,PTON,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2016/01/05,23.376617,48.921055,31.689501,38.076500,37.129002,14.895333,102.730003,8.027896,34.310001,107.660004,...,NaN,47.689999,30.877262,103.449997,182.669998,114.849823,19.973000,3.475622,24.208334,51.900974
2016/01/06,22.919142,48.032391,31.632500,37.966499,37.181000,14.602667,102.970001,7.695942,33.980000,117.680000,...,NaN,47.889999,30.843512,100.440002,183.210007,113.997566,19.487000,3.457844,24.477110,50.744545
2016/01/07,21.951851,46.361698,30.396999,37.049999,36.319500,14.376667,97.919998,7.390838,33.130001,114.559998,...,NaN,45.250000,29.369446,97.089996,182.979996,109.946747,18.429001,3.413398,23.809803,49.907436
2016/01/08,22.067924,46.503891,30.352501,36.545502,35.723499,14.066667,97.330002,7.232183,32.689999,111.389999,...,NaN,45.029999,29.030231,95.750000,182.169998,106.690742,18.212000,3.386731,23.865412,47.801704
2016/01/11,22.425255,46.477219,30.886999,36.653500,35.801498,13.856667,97.510002,7.244387,33.040001,114.970001,...,NaN,43.619999,28.200846,93.290001,182.000000,106.489632,18.011999,3.395620,24.189802,45.566536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/12/23,128.687256,215.239517,159.263504,86.411499,86.619003,215.326660,268.109985,129.811371,239.440002,514.479980,...,162.759995,32.630001,89.001801,88.300003,268.459991,126.489998,90.775002,5.767800,48.795238,49.930000
2020/12/24,129.679718,216.924255,158.634506,86.708000,86.942497,220.589996,267.399994,129.656693,238.639999,513.969971,...,162.720001,31.990000,89.078705,88.660004,264.640015,126.959999,90.663002,5.804596,47.831356,49.919998
2020/12/28,134.317825,219.076462,164.197998,88.697998,88.804497,221.229996,277.000000,128.721222,235.729996,519.119995,...,152.179993,31.900000,87.607826,87.580002,278.170013,128.479996,93.949997,5.841393,47.945881,50.060001


In [6]:
training_log_ret=training_log_ret.iloc[1:, :]

In [7]:
training_log_ret

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,PTON,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2016/01/06,-0.019764,-0.018332,-0.001800,-0.002893,0.001399,-0.019844,0.002333,-0.042229,-0.009665,0.088991,...,NaN,0.004185,-0.001094,-0.029528,0.002952,-0.007448,-0.024634,-0.005128,0.011041,-0.022533
2016/01/07,-0.043121,-0.035402,-0.039841,-0.024436,-0.023443,-0.015598,-0.050287,-0.040452,-0.025333,-0.026870,...,NaN,-0.056704,-0.048972,-0.033922,-0.001256,-0.036181,-0.055822,-0.012937,-0.027641,-0.016634
2016/01/08,0.005274,0.003062,-0.001465,-0.013710,-0.016546,-0.021799,-0.006044,-0.021700,-0.013370,-0.028061,...,NaN,-0.004874,-0.011617,-0.013898,-0.004437,-0.030062,-0.011845,-0.007843,0.002333,-0.043109
2016/01/11,0.016063,-0.000574,0.017456,0.002951,0.002181,-0.015041,0.001848,0.001686,0.010650,0.031634,...,NaN,-0.031813,-0.028986,-0.026028,-0.000934,-0.001887,-0.011043,0.002621,0.013501,-0.047888
2016/01/12,0.014409,0.009136,0.000243,0.016599,0.013924,0.010148,0.018895,0.016706,0.001210,0.013907,...,NaN,0.021994,0.012952,-0.001931,0.019695,0.006275,0.028191,-0.007884,0.004205,-0.008368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/12/23,-0.007001,-0.013125,-0.006649,0.004646,0.005139,0.008769,0.003812,-0.020467,-0.016773,-0.024670,...,0.009569,-0.009759,0.000865,-0.000566,0.012633,0.026760,0.002670,0.000000,-0.009538,0.004215
2020/12/24,0.007683,0.007797,-0.003957,0.003425,0.003728,0.024150,-0.002652,-0.001192,-0.003347,-0.000992,...,-0.000246,-0.019809,0.000864,0.004069,-0.014331,0.003709,-0.001235,0.006359,-0.019951,-0.000200
2020/12/28,0.035141,0.009873,0.034470,0.022691,0.021190,0.002897,0.035272,-0.007241,-0.012269,0.009970,...,-0.066967,-0.002817,-0.016650,-0.012256,0.049862,0.011901,0.035613,0.006319,0.002391,0.002801


In [8]:
training_log_ret.isna().sum().ge(100).value_counts()

False    91
True      9
dtype: int64

In [9]:
training_log_ret.isna().sum().value_counts()

0       91
645      1
828      1
583      1
1194     1
801      1
802      1
318      1
181      1
939      1
dtype: int64

In [10]:
drop_lst=['PDD', 'ZM', 'DOCU', 'ASAN', 'FOXA', 'FOX', 'OKTA', 'TTD', 'PTON']

In [11]:
training_log_ret=training_log_ret.drop(columns=drop_lst)

In [82]:
alpha=pd.read_csv('first round alpha.csv', header=0)
alpha=alpha.set_index(['ticker', 'Date'])

In [83]:
alpha.replace([np.inf, -np.inf], np.nan, inplace=True)
alpha=alpha.ffill()
alpha=alpha.groupby('ticker').transform(lambda x: x.fillna(x.mean()))

In [84]:
alpha

a51        a49        a12         a35       a54  \
ticker Date                                                                
AAPL   2016/01/05  -1.000000  -1.000000   0.079023  2571.71717 -0.100901   
       2016/01/06  -1.000000  -1.000000   0.502501  2571.71717 -0.329696   
       2016/01/07  -1.000000  -1.000000   1.062500  2571.71717 -0.006059   
       2016/01/08  -1.000000  -1.000000   0.127501  2571.71717 -0.092316   
       2016/01/11  -1.000000  -1.000000   0.392500  2571.71717 -0.707448   
...                      ...        ...        ...         ...       ...   
ZM     2020/12/23  25.119995  25.119995  25.119995    -0.00000 -0.277880   
       2020/12/24   8.709991   8.709991  -8.709991    -0.00000 -0.019300   
       2020/12/28  23.779999  23.779999  23.779999    -0.00000 -0.127136   
       2020/12/29  -2.359985  -2.359985   2.359985    -0.00000 -0.354365   
       2020/12/30   0.350006   0.350006  -0.350006   165.00000 -0.645456   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2016/01/05  0.483688 -0.149385 -0.292017 -0.454874  0.508811  ...   
       2016/01/06  0.483688 -0.149385 -0.292017 -0.454874  0.508811  ...   
       2016/01/07  0.483688 -0.149385 -0.292017 -0.454874  0.508811  ...   
       2016/01/08  0.483688 -0.149385 -0.292017 -0.454874  0.508811  ...   
       2016/01/11  0.483688 -0.149385 -0.292017 -0.454874  0.439560  ...   
...                     ...       ...       ...       ...       ...  ...   
ZM     2020/12/23  0.990000 -0.001287 -0.015800 -0.020000  0.655000  ...   
       2020/12/24  0.990000 -0.019206 -0.000350 -0.040000  0.810000  ...   
       2020/12/28  1.000000 -0.002489 -0.001050 -0.020000  0.865000  ...   
       2020/12/29  0.060000 -0.089743 -0.046000 -0.960000  0.050000  ...   
       2020/12/30  0.880000 -0.008649 -0.009800 -0.770000  0.905000  ...   

                        a5       a45  a23  a74       a13       a84  a81  \
ticker Date                                                               
AAPL   2016/01/05  0.25104  0.008083  0.0    0 -0.503709  0.493383    0   
       2016/01/06  0.25104  0.008083  0.0    0 -0.503709  0.493383    0   
       2016/01/07  0.25104  0.008083  0.0    0 -0.503709  0.493383    0   
       2016/01/08  0.25104  0.008083  0.0    0 -0.503709  0.493383    0   
       2016/01/11  0.25104  0.008083  0.0    0 -0.664835  0.493383    0   
...                    ...       ...  ...  ...       ...       ...  ...   
ZM     2020/12/23  0.01710  0.648000  0.0    0 -0.555000  0.010000    0   
       2020/12/24  0.00020 -0.657000  0.0    0 -0.060000  0.010000    0   
       2020/12/28  0.00010  0.679500  0.0    0 -0.050000  0.010000    0   
       2020/12/29  0.00840  0.666000  0.0    0 -0.040000  0.010000    0   
       2020/12/30  0.00250 -0.711000  0.0    0 -0.860000  0.010000   -1   

                        a11  a21       a14  
ticker Date                                 
AAPL   2016/01/05  0.469912    1  0.017003  
       2016/01/06  0.469912    1  0.017003  
       2016/01/07  0.469912    1  0.017003  
       2016/01/08  1.010989    1  0.017003  
       2016/01/11  0.009057    1  0.017003  
...                     ...  ...       ...  
ZM     2020/12/23  1.227600    1 -0.004774  
       2020/12/24  0.271400    1 -0.002961  
       2020/12/28  1.881000    1  0.005598  
       2020/12/29  0.085000    1  0.107946  
       2020/12/30  1.206200    1  0.036829  

[125700 rows x 30 columns]

alpha_nor = alpha.groupby('ticker').transform(lambda x: (x-x.mean())/x.std(ddof=1))
alpha_nor

In [15]:
for t in drop_lst:
    alpha.drop(t, level=0, axis=0, inplace=True) 

In [16]:
alpha=alpha[alpha.index.get_level_values('Date') >='2016/02/03']

In [17]:
alpha

a51       a49       a12         a35       a54  \
ticker Date                                                             
AAPL   2016/02/03 -0.467499 -0.467499 -0.467499  2571.71717 -0.766437   
       2016/02/04 -0.062500 -0.062500 -0.062500  2571.71717 -0.634024   
       2016/02/05  0.645000  0.645000 -0.645000  2571.71717 -0.116490   
       2016/02/08 -0.247501 -0.247501 -0.247501  2571.71717 -0.670173   
       2016/02/09  0.005001  0.005001 -0.005001  2571.71717 -0.508215   
...                     ...       ...       ...         ...       ...   
ZBRA   2020/12/23  0.220001  0.220001 -0.220001   286.00000 -0.333551   
       2020/12/24  2.849976  2.849976 -2.849976    99.00000 -0.541726   
       2020/12/28 -5.699982 -5.699982 -5.699982  2925.00000 -0.806574   
       2020/12/29  4.549988  4.549988 -4.549988   162.00000 -0.198604   
       2020/12/30 -5.709991 -5.709991  5.709991   858.00000 -0.659492   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2016/02/03  0.054945 -0.149385 -0.308296 -0.648352  0.247253  ...   
       2016/02/04  0.197802 -0.149385 -0.279435 -0.428571  0.538462  ...   
       2016/02/05  0.714286 -0.149385 -0.414080 -0.692308  0.329670  ...   
       2016/02/08  0.032967 -0.460193 -0.316870 -0.648352  0.032967  ...   
       2016/02/09  0.549451 -0.029111 -0.312040 -0.043956  0.609890  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2020/12/23  0.570000 -0.060501 -0.286700 -0.540000  0.370000  ...   
       2020/12/24  0.890000 -0.012658 -0.028000 -0.020000  0.940000  ...   
       2020/12/28  0.160000 -0.319680 -0.276500 -0.910000  0.020000  ...   
       2020/12/29  0.880000 -0.008167 -0.141750 -0.020000  0.640000  ...   
       2020/12/30  0.180000 -0.157140 -0.382200 -0.880000  0.045000  ...   

                         a5       a45       a23  a74       a13       a84  a81  \
ticker Date                                                                     
AAPL   2016/02/03  0.240913  0.008083  0.000000    0 -0.428571  0.493383    0   
       2016/02/04  0.239102  0.008083  0.000000    0 -0.417582  0.493383    0   
       2016/02/05  0.375679  0.008083  0.000000    0 -0.527473  0.493383    0   
       2016/02/08  0.548726  0.008083  0.000000    0 -0.527473  0.493383    0   
       2016/02/09  0.425069 -0.028257  0.000000    0 -0.692308  0.493383    0   
...                     ...       ...       ...  ...       ...       ...  ...   
ZBRA   2020/12/23  0.208800 -0.339300 -7.869995    0 -0.120000  0.080000   -1   
       2020/12/24  0.053200 -0.348000  5.000000    0 -0.160000  0.290000    0   
       2020/12/28  0.533600 -0.495900  0.600006    0 -0.250000  0.860000    0   
       2020/12/29  0.017400 -0.287100 -3.130005    0 -0.270000  0.860000    0   
       2020/12/30  0.682500  0.487200 -1.760010    0 -0.660000  0.890000    0   

                        a11  a21       a14  
ticker Date                                 
AAPL   2016/02/03  0.021012    1 -0.032780  
       2016/02/04  0.825987    1 -0.046626  
       2016/02/05  0.849052    1 -0.032190  
       2016/02/08  0.802198    1 -0.144546  
       2016/02/09  0.047096    1 -0.072229  
...                     ...  ...       ...  
ZBRA   2020/12/23  0.643800    1  0.085961  
       2020/12/24  1.465100    1  0.009409  
       2020/12/28  0.390000    1  0.318392  
       2020/12/29  0.558600    1  0.013894  
       2020/12/30  0.078400    1  0.439637  

[112567 rows x 30 columns]

In [18]:
from sklearn.preprocessing import scale
alpha_nor=alpha.groupby('ticker').transform(lambda x: scale(x))
alpha_nor

a51       a49       a12           a35       a54  \
ticker Date                                                               
AAPL   2016/02/03 -0.288753 -0.288753 -0.416899 -1.327336e-16 -0.865137   
       2016/02/04  0.020221  0.020221 -0.108073 -1.327336e-16 -0.394148   
       2016/02/05  0.559974  0.559974 -0.552250 -1.327336e-16  1.446707   
       2016/02/08 -0.120916 -0.120916 -0.249143 -1.327336e-16 -0.522728   
       2016/02/09  0.071718  0.071718 -0.064228 -1.327336e-16  0.053353   
...                     ...       ...       ...           ...       ...   
ZBRA   2020/12/23  0.110132  0.110132 -0.061921 -6.310006e-01  0.612775   
       2020/12/24  0.709318  0.709318 -0.660073 -6.865912e-01 -0.143807   
       2020/12/28 -1.238615 -1.238615 -1.308269  1.535110e-01 -1.106361   
       2020/12/29  1.096631  1.096631 -1.046718 -6.678628e-01  1.103220   
       2020/12/30 -1.240896 -1.240896  1.286776 -4.609587e-01 -0.571810   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2016/02/03 -1.285677  0.000000 -0.074604 -1.020275 -0.994222  ...   
       2016/02/04 -0.857112  0.000000  0.064509  0.140747  0.115370  ...   
       2016/02/05  0.692314  0.000000 -0.584489 -1.252479 -0.680186  ...   
       2016/02/08 -1.351610 -2.008652 -0.115930 -1.020275 -1.810714  ...   
       2016/02/09  0.197816  0.777291 -0.092648  2.172536  0.387534  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2020/12/23  0.265192  0.440153 -0.120103  0.166178 -0.399777  ...   
       2020/12/24  1.567567  0.702366  1.015162  1.748807  1.456013  ...   
       2020/12/28 -1.403476 -0.980344 -0.075342 -0.959923 -1.539297  ...   
       2020/12/29  1.526868  0.726980  0.515987  1.748807  0.479281  ...   
       2020/12/30 -1.322078 -0.089502 -0.539190 -0.868618 -1.457903  ...   

                         a5           a45       a23       a74       a13  \
ticker Date                                                               
AAPL   2016/02/03 -0.069897 -2.852414e-17  0.264835  1.087041  0.451078   
       2016/02/04 -0.082496 -2.852414e-17  0.264835  1.087041  0.516681   
       2016/02/05  0.867487 -2.852414e-17  0.264835  1.087041 -0.139350   
       2016/02/08  2.071135 -2.852414e-17  0.264835  1.087041 -0.139350   
       2016/02/09  1.211026 -1.991817e-01  0.264835  1.087041 -1.123395   
...                     ...           ...       ...       ...       ...   
ZBRA   2020/12/23 -0.257603 -1.002152e+00 -1.511068  0.803086  1.570189   
       2020/12/24 -0.788642 -1.028097e+00  1.425119  0.803086  1.418142   
       2020/12/28  0.850890 -1.469155e+00  0.421295  0.803086  1.076036   
       2020/12/29 -0.910822 -8.464846e-01 -0.429677  0.803086  1.000012   
       2020/12/30  1.359062  1.462586e+00 -0.117124  0.803086 -0.482448   

                            a84       a81       a11       a21       a14  
ticker Date                                                              
AAPL   2016/02/03  5.460353e-16  0.757490 -0.915863  0.629414 -0.213018  
       2016/02/04  5.460353e-16  0.757490  0.727112  0.629414 -0.274093  
       2016/02/05  5.460353e-16  0.757490  0.774188  0.629414 -0.210418  
       2016/02/08  5.460353e-16  0.757490  0.678557  0.629414 -0.705986  
       2016/02/09  5.460353e-16  0.757490 -0.862625  0.629414 -0.387017  
...                         ...       ...       ...       ...       ...  
ZBRA   2020/12/23 -1.417555e+00 -1.327087  0.483595  0.553381  0.269060  
       2020/12/24 -7.869338e-01  0.753530  3.238359  0.553381 -0.034703  
       2020/12/28  9.247524e-01  0.753530 -0.367688  0.553381  1.191359  
       2020/12/29  9.247524e-01  0.753530  0.197822  0.553381 -0.016905  
       2020/12/30  1.014841e+00  0.753530 -1.412842  0.553381  1.672465  

[112567 rows x 30 columns]

In [19]:
alpha_nor.isna().sum()

a51    0
a49    0
a12    0
a35    0
a54    0
a25    0
a17    0
a38    0
a18    0
a34    0
a46    0
a42    0
a28    0
a83    0
a60    0
a7     0
a47    0
a43    0
a20    0
a24    0
a5     0
a45    0
a23    0
a74    0
a13    0
a84    0
a81    0
a11    0
a21    0
a14    0
dtype: int64

In [20]:
alpha_nor.loc[['ZBRA']]

a51       a49       a12       a35       a54       a25  \
ticker Date                                                                     
ZBRA   2016/02/03 -0.340970 -0.340970  0.388403 -0.585220 -1.328388 -0.355135   
       2016/02/04 -0.240726 -0.240726 -0.312101 -0.585220 -0.077067 -1.383797   
       2016/02/05  0.887030  0.887030  0.813709 -0.585220  1.442462  0.181558   
       2016/02/08  0.729829  0.729829 -0.680549 -0.585220  0.785915  0.405180   
       2016/02/09  0.005330  0.005330  0.042700 -0.585220 -1.063040 -0.578758   
...                     ...       ...       ...       ...       ...       ...   
       2020/12/23  0.110132  0.110132 -0.061921 -0.631001  0.612775  0.265192   
       2020/12/24  0.709318  0.709318 -0.660073 -0.686591 -0.143807  1.567567   
       2020/12/28 -1.238615 -1.238615 -1.308269  0.153511 -1.106361 -1.403476   
       2020/12/29  1.096631  1.096631 -1.046718 -0.667863  1.103220  1.526868   
       2020/12/30 -1.240896 -1.240896  1.286776 -0.460959 -0.571810 -1.322078   

                        a17       a38       a18       a34  ...        a5  \
ticker Date                                                ...             
ZBRA   2016/02/03  0.149898 -1.368530 -1.033169 -0.906747  ...  1.073546   
       2016/02/04  0.149898 -2.001795 -1.033169 -0.101749  ...  1.505046   
       2016/02/05  0.149898  0.466350  1.241109  0.184473  ... -0.199935   
       2016/02/08  0.633324  0.347911  1.374890  1.257804  ... -0.647096   
       2016/02/09  0.484643  0.001867 -0.163592 -0.495304  ... -0.049920   
...                     ...       ...       ...       ...  ...       ...   
       2020/12/23  0.440153 -0.120103  0.166178 -0.399777  ... -0.257603   
       2020/12/24  0.702366  1.015162  1.748807  1.456013  ... -0.788642   
       2020/12/28 -0.980344 -0.075342 -0.959923 -1.539297  ...  0.850890   
       2020/12/29  0.726980  0.515987  1.748807  0.479281  ... -0.910822   
       2020/12/30 -0.089502 -0.539190 -0.868618 -1.457903  ...  1.359062   

                        a45       a23       a74       a13       a84       a81  \
ticker Date                                                                     
ZBRA   2016/02/03  0.252434  0.448670  0.803086  0.773194 -0.126283  0.753530   
       2016/02/04  0.252434 -0.363515  0.803086 -0.897654 -0.126283  0.753530   
       2016/02/05  0.252434  0.022046  0.803086 -0.020459 -0.126283  0.753530   
       2016/02/08  0.252434  0.284408  0.803086  0.397253 -0.126283  0.753530   
       2016/02/09  0.925829  0.284408  0.803086 -1.022968 -0.126283  0.753530   
...                     ...       ...       ...       ...       ...       ...   
       2020/12/23 -1.002152 -1.511068  0.803086  1.570189 -1.417555 -1.327087   
       2020/12/24 -1.028097  1.425119  0.803086  1.418142 -0.786934  0.753530   
       2020/12/28 -1.469155  0.421295  0.803086  1.076036  0.924752  0.753530   
       2020/12/29 -0.846485 -0.429677  0.803086  1.000012  0.924752  0.753530   
       2020/12/30  1.462586 -0.117124  0.803086 -0.482448  1.014841  0.753530   

                        a11       a21       a14  
ticker Date                                      
ZBRA   2016/02/03 -0.101815  0.553381  0.131069  
       2016/02/04 -0.514552  0.553381  0.259351  
       2016/02/05 -0.822789  0.553381  0.050947  
       2016/02/08 -0.432734  0.553381 -0.125712  
       2016/02/09 -0.653887  0.553381 -0.463470  
...                     ...       ...       ...  
       2020/12/23  0.483595  0.553381  0.269060  
       2020/12/24  3.238359  0.553381 -0.034703  
       2020/12/28 -0.367688  0.553381  1.191359  
       2020/12/29  0.197822  0.553381 -0.016905  
       2020/12/30 -1.412842  0.553381  1.672465  

[1237 rows x 30 columns]

# Lasso

In [22]:
alpha_test_2021=pd.read_csv('backtesting alpha 2021.csv', header=0)
alpha_test_2021=alpha_test_2021.set_index(['ticker', 'Date'])

alpha_test_2021_2022=pd.read_csv('backtesting alpha 2021&2022.csv', header=0)
alpha_test_2021_2022=alpha_test_2021_2022.set_index(['ticker', 'Date'])

In [23]:
alpha_test_2021

a51        a49        a12      a35       a54  \
ticker Date                                                             
AAPL   2021/01/05  -1.000000  -1.000000        NaN      NaN -0.718397   
       2021/01/06  -1.000000  -1.000000   4.409996      NaN -0.049230   
       2021/01/07  -1.000000  -1.000000   4.320000      NaN -0.735356   
       2021/01/08  -1.000000  -1.000000   1.130005      NaN -0.769307   
       2021/01/11  -1.000000  -1.000000  -3.070007      NaN -0.289770   
...                      ...        ...        ...      ...       ...   
ZBRA   2021/12/23  -1.979980  -1.979980  -1.979980   1302.0 -0.459641   
       2021/12/27 -23.920044 -23.920044 -23.920044  14880.0 -0.809768   
       2021/12/28   9.010010   9.010010  -9.010010    160.0 -0.229235   
       2021/12/29  -3.799988  -3.799988   3.799988    294.0 -0.639683   
       2021/12/30   3.580017   3.580017  -3.580017     78.0 -0.154421   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2021/01/05       NaN       NaN       NaN       NaN       NaN  ...   
       2021/01/06       NaN       NaN       NaN       NaN       NaN  ...   
       2021/01/07       NaN       NaN       NaN       NaN       NaN  ...   
       2021/01/08       NaN       NaN       NaN       NaN       NaN  ...   
       2021/01/11       NaN       NaN       NaN       NaN  0.846154  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2021/12/23  0.307692 -0.042040 -0.016906 -0.285714  0.675824  ...   
       2021/12/27  0.197802 -0.627943 -0.058447 -0.978022  0.043956  ...   
       2021/12/28  0.835165 -0.000920 -0.123657 -0.241758  0.516484  ...   
       2021/12/29  0.186813 -0.076886 -0.266997 -0.967033  0.175824  ...   
       2021/12/30  0.780220 -0.004802 -0.079882 -0.791209  0.956044  ...   

                         a5       a45        a23  a74       a13       a84  \
ticker Date                                                                 
AAPL   2021/01/05       NaN       NaN   0.000000    0       NaN       NaN   
       2021/01/06       NaN       NaN   0.000000    0       NaN       NaN   
       2021/01/07       NaN       NaN   0.000000    0       NaN       NaN   
       2021/01/08       NaN       NaN   0.000000    0       NaN       NaN   
       2021/01/11       NaN       NaN   0.000000    0 -0.631868       NaN   
...                     ...       ...        ...  ...       ...       ...   
ZBRA   2021/12/23  0.004830 -0.743872   0.000000    0 -0.054945  0.087912   
       2021/12/27  0.054945 -0.763193 -24.800049    0 -0.434066  0.703297   
       2021/12/28  0.056515 -0.096607 -22.460022    0 -0.626374  0.626374   
       2021/12/29  0.639053  0.859585   3.750000    0 -0.450549  0.835165   
       2021/12/30  0.073421 -0.696413   3.830017    0 -0.626374  0.846154   

                   a81       a11  a21       a14  
ticker Date                                      
AAPL   2021/01/05    0       NaN    1       NaN  
       2021/01/06    0       NaN    1       NaN  
       2021/01/07    0       NaN    1       NaN  
       2021/01/08    0  1.188262    1       NaN  
       2021/01/11    0  0.009419    1       NaN  
...                ...       ...  ...       ...  
ZBRA   2021/12/23    0  1.470837    1  0.524067  
       2021/12/27   -1  0.792296    1  0.456526  
       2021/12/28    0  0.567323   -1  0.115708  
       2021/12/29    0  0.405024    1  0.584279  
       2021/12/30    0  0.331361    1  0.015954  

[22750 rows x 30 columns]

In [49]:
alpha_test_2021=alpha_test_2021[alpha_test_2021.index.get_level_values('Date') >='2021/02/03']
alpha_test_2021_2022=alpha_test_2021_2022[alpha_test_2021_2022.index.get_level_values('Date') >='2021/02/03']

In [50]:
alpha_test_2021

a51        a49        a12           a35       a54  \
ticker Date                                                                  
AAPL   2021/02/03   1.050003   1.050003   1.050003   2490.237443 -0.163444   
       2021/02/04  -3.449997  -3.449997   3.449997   2490.237443 -0.957539   
       2021/02/05   0.630005   0.630005  -0.630005   2490.237443 -0.589473   
       2021/02/08  -0.150009  -0.150009   0.150009   2490.237443 -0.944539   
       2021/02/09   0.900009   0.900009   0.900009   2490.237443 -0.080595   
...                      ...        ...        ...           ...       ...   
ZBRA   2021/12/23  -1.979980  -1.979980  -1.979980   1302.000000 -0.459641   
       2021/12/27 -23.920044 -23.920044 -23.920044  14880.000000 -0.809768   
       2021/12/28   9.010010   9.010010  -9.010010    160.000000 -0.229235   
       2021/12/29  -3.799988  -3.799988   3.799988    294.000000 -0.639683   
       2021/12/30   3.580017   3.580017  -3.580017     78.000000 -0.154421   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2021/02/03  0.802198 -0.137292 -0.084169 -0.527473  0.901099  ...   
       2021/02/04  0.373626 -0.137292 -0.197802 -0.538462  0.412088  ...   
       2021/02/05  0.890110 -0.137292 -0.101497 -0.406593  0.219780  ...   
       2021/02/08  0.472527 -0.018434 -0.121724 -0.538462  0.747253  ...   
       2021/02/09  0.956044 -0.051722 -0.099746 -0.175824  0.857143  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2021/12/23  0.307692 -0.042040 -0.016906 -0.285714  0.675824  ...   
       2021/12/27  0.197802 -0.627943 -0.058447 -0.978022  0.043956  ...   
       2021/12/28  0.835165 -0.000920 -0.123657 -0.241758  0.516484  ...   
       2021/12/29  0.186813 -0.076886 -0.266997 -0.967033  0.175824  ...   
       2021/12/30  0.780220 -0.004802 -0.079882 -0.791209  0.956044  ...   

                         a5       a45        a23  a74       a13       a84  \
ticker Date                                                                 
AAPL   2021/02/03  0.051926 -0.000218  -0.389999    0 -0.813187  0.517015   
       2021/02/04  0.147084 -0.000218  -1.089996    0 -0.813187  0.517015   
       2021/02/05  0.085135 -0.000218  -1.649994    0 -0.780220  0.517015   
       2021/02/08  0.104818 -0.000218   0.439987    0 -0.725275  0.517015   
       2021/02/09  0.040575  0.294590  -0.460007    0 -0.725275  0.517015   
...                     ...       ...        ...  ...       ...       ...   
ZBRA   2021/12/23  0.004830 -0.743872   0.000000    0 -0.054945  0.087912   
       2021/12/27  0.054945 -0.763193 -24.800049    0 -0.434066  0.703297   
       2021/12/28  0.056515 -0.096607 -22.460022    0 -0.626374  0.626374   
       2021/12/29  0.639053  0.859585   3.750000    0 -0.450549  0.835165   
       2021/12/30  0.073421 -0.696413   3.830017    0 -0.626374  0.846154   

                   a81       a11  a21       a14  
ticker Date                                      
AAPL   2021/02/03    0  0.013404    1 -0.224561  
       2021/02/04    0  0.023910    1 -0.264246  
       2021/02/05    0  0.070161    1 -0.173221  
       2021/02/08    0  0.030914    1 -0.094295  
       2021/02/09    0  0.082840    1 -0.091278  
...                ...       ...  ...       ...  
ZBRA   2021/12/23    0  1.470837    1  0.524067  
       2021/12/27   -1  0.792296    1  0.456526  
       2021/12/28    0  0.567323   -1  0.115708  
       2021/12/29    0  0.405024    1  0.584279  
       2021/12/30    0  0.331361    1  0.015954  

[20930 rows x 30 columns]

In [51]:
alpha_test_2021=alpha_test_2021[alpha_nor.columns]
alpha_test_2021_2022=alpha_test_2021_2022[alpha_nor.columns]

In [52]:
alpha_test_2021

a51        a49        a12           a35       a54  \
ticker Date                                                                  
AAPL   2021/02/03   1.050003   1.050003   1.050003   2490.237443 -0.163444   
       2021/02/04  -3.449997  -3.449997   3.449997   2490.237443 -0.957539   
       2021/02/05   0.630005   0.630005  -0.630005   2490.237443 -0.589473   
       2021/02/08  -0.150009  -0.150009   0.150009   2490.237443 -0.944539   
       2021/02/09   0.900009   0.900009   0.900009   2490.237443 -0.080595   
...                      ...        ...        ...           ...       ...   
ZBRA   2021/12/23  -1.979980  -1.979980  -1.979980   1302.000000 -0.459641   
       2021/12/27 -23.920044 -23.920044 -23.920044  14880.000000 -0.809768   
       2021/12/28   9.010010   9.010010  -9.010010    160.000000 -0.229235   
       2021/12/29  -3.799988  -3.799988   3.799988    294.000000 -0.639683   
       2021/12/30   3.580017   3.580017  -3.580017     78.000000 -0.154421   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2021/02/03  0.802198 -0.137292 -0.084169 -0.527473  0.901099  ...   
       2021/02/04  0.373626 -0.137292 -0.197802 -0.538462  0.412088  ...   
       2021/02/05  0.890110 -0.137292 -0.101497 -0.406593  0.219780  ...   
       2021/02/08  0.472527 -0.018434 -0.121724 -0.538462  0.747253  ...   
       2021/02/09  0.956044 -0.051722 -0.099746 -0.175824  0.857143  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2021/12/23  0.307692 -0.042040 -0.016906 -0.285714  0.675824  ...   
       2021/12/27  0.197802 -0.627943 -0.058447 -0.978022  0.043956  ...   
       2021/12/28  0.835165 -0.000920 -0.123657 -0.241758  0.516484  ...   
       2021/12/29  0.186813 -0.076886 -0.266997 -0.967033  0.175824  ...   
       2021/12/30  0.780220 -0.004802 -0.079882 -0.791209  0.956044  ...   

                         a5       a45        a23  a74       a13       a84  \
ticker Date                                                                 
AAPL   2021/02/03  0.051926 -0.000218  -0.389999    0 -0.813187  0.517015   
       2021/02/04  0.147084 -0.000218  -1.089996    0 -0.813187  0.517015   
       2021/02/05  0.085135 -0.000218  -1.649994    0 -0.780220  0.517015   
       2021/02/08  0.104818 -0.000218   0.439987    0 -0.725275  0.517015   
       2021/02/09  0.040575  0.294590  -0.460007    0 -0.725275  0.517015   
...                     ...       ...        ...  ...       ...       ...   
ZBRA   2021/12/23  0.004830 -0.743872   0.000000    0 -0.054945  0.087912   
       2021/12/27  0.054945 -0.763193 -24.800049    0 -0.434066  0.703297   
       2021/12/28  0.056515 -0.096607 -22.460022    0 -0.626374  0.626374   
       2021/12/29  0.639053  0.859585   3.750000    0 -0.450549  0.835165   
       2021/12/30  0.073421 -0.696413   3.830017    0 -0.626374  0.846154   

                   a81       a11  a21       a14  
ticker Date                                      
AAPL   2021/02/03    0  0.013404    1 -0.224561  
       2021/02/04    0  0.023910    1 -0.264246  
       2021/02/05    0  0.070161    1 -0.173221  
       2021/02/08    0  0.030914    1 -0.094295  
       2021/02/09    0  0.082840    1 -0.091278  
...                ...       ...  ...       ...  
ZBRA   2021/12/23    0  1.470837    1  0.524067  
       2021/12/27   -1  0.792296    1  0.456526  
       2021/12/28    0  0.567323   -1  0.115708  
       2021/12/29    0  0.405024    1  0.584279  
       2021/12/30    0  0.331361    1  0.015954  

[20930 rows x 30 columns]

In [53]:
alpha_test_2021.replace([np.inf, -np.inf], np.nan, inplace=True)
alpha_test_2021=alpha_test_2021.ffill()
alpha_test_2021=alpha_test_2021.groupby('ticker').transform(lambda x: x.fillna(x.mean()))

alpha_test_2021_2022.replace([np.inf, -np.inf], np.nan, inplace=True)
alpha_test_2021_2022=alpha_test_2021_2022.ffill()
alpha_test_2021_2022=alpha_test_2021_2022.groupby('ticker').transform(lambda x: x.fillna(x.mean()))

In [54]:
alpha_test_nor_2021=alpha_test_2021.groupby('ticker').transform(lambda x: scale(x))
alpha_test_nor_2021_2022=alpha_test_2021_2022.groupby('ticker').transform(lambda x: scale(x))
alpha_test_nor_2021

a51       a49       a12       a35       a54       a25  \
ticker Date                                                                     
AAPL   2021/02/03  0.581220  0.581220  0.377991  0.000000  1.161355  0.826532   
       2021/02/04 -1.531671 -1.531671  1.508087  0.000000 -1.513942 -0.304167   
       2021/02/05  0.384018  0.384018 -0.413082  0.000000 -0.273932  1.058471   
       2021/02/08  0.017777  0.017777 -0.045793  0.000000 -1.470146 -0.043236   
       2021/02/09  0.510793  0.510793  0.307363  0.000000  1.440472  1.232424   
...                     ...       ...       ...       ...       ...       ...   
ZBRA   2021/12/23 -0.124594 -0.124594 -0.168311 -0.297554  0.199164 -0.693614   
       2021/12/27 -2.514773 -2.514773 -2.551230  3.961733 -1.057598 -1.068982   
       2021/12/28  1.072670  1.072670 -0.931845 -0.655789  1.026192  1.108150   
       2021/12/29 -0.322868 -0.322868  0.459454 -0.613754 -0.447087 -1.106518   
       2021/12/30  0.481120  0.481120 -0.342091 -0.681511  1.294734  0.920467   

                        a17       a38       a18       a34  ...        a5  \
ticker Date                                                ...             
AAPL   2021/02/03  0.000000  0.894630 -0.147203  1.480758  ... -1.285006   
       2021/02/04  0.000000  0.368820 -0.203537 -0.316628  ... -0.704244   
       2021/02/05  0.000000  0.814446  0.472471 -1.023465  ... -1.082329   
       2021/02/08  0.805405  0.720850 -0.203537  0.915288  ... -0.962197   
       2021/02/09  0.579842  0.822548  1.655487  1.319195  ... -1.354285   
...                     ...       ...       ...       ...  ...       ...   
ZBRA   2021/12/23  0.558769  1.170582  0.871201  0.505285  ... -0.899652   
       2021/12/27 -2.834756  0.975725 -1.026815 -1.413289  ... -0.753283   
       2021/12/28  0.796935  0.669845  0.991710  0.021471  ... -0.748698   
       2021/12/29  0.356942 -0.002524 -0.996688 -1.012891  ...  0.952717   
       2021/12/30  0.774446  0.875181 -0.514652  1.356130  ... -0.699320   

                            a45       a23       a74       a13           a84  \
ticker Date                                                                   
AAPL   2021/02/03  3.634981e-17  0.211120  0.744348 -1.415909 -1.045026e-15   
       2021/02/04  3.634981e-17 -0.119427  0.744348 -1.415909 -1.045026e-15   
       2021/02/05  3.634981e-17 -0.383865  0.744348 -1.240273 -1.045026e-15   
       2021/02/08  3.634981e-17  0.603051  0.744348 -0.947546 -1.045026e-15   
       2021/02/09  9.834800e-01  0.178062  0.744348 -0.947546 -1.045026e-15   
...                         ...       ...       ...       ...           ...   
ZBRA   2021/12/23 -1.232189e+00  0.349405  0.675053  2.608270 -1.359833e+00   
       2021/12/27 -1.264718e+00 -2.240228  0.675053  0.503778  4.140719e-01   
       2021/12/28 -1.424707e-01 -1.995882  0.675053 -0.563717  1.923337e-01   
       2021/12/29  1.467349e+00  0.740981  0.675053  0.412279  7.941944e-01   
       2021/12/30 -1.152290e+00  0.749337  0.675053 -0.563717  8.258712e-01   

                        a81       a11       a21       a14  
ticker Date                                                
AAPL   2021/02/03  0.838870 -0.861017  0.540495 -1.125624  
       2021/02/04  0.838870 -0.839307  0.540495 -1.290649  
       2021/02/05  0.838870 -0.743732  0.540495 -0.912139  
       2021/02/08  0.838870 -0.824833  0.540495 -0.583938  
       2021/02/09  0.838870 -0.717530  0.540495 -0.571394  
...                     ...       ...       ...       ...  
ZBRA   2021/12/23  0.574003  2.800034  0.587392  1.540104  
       2021/12/27 -1.742150  0.796750  0.587392  1.318605  
       2021/12/28  0.574003  0.132554 -1.702441  0.200903  
       2021/12/29  0.574003 -0.346608  0.587392  1.737567  
       2021/12/30  0.574003 -0.564085  0.587392 -0.126239  

[20930 rows x 30 columns]

In [55]:
tickers=["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "ANSS",
    "SWKS", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]


In [56]:
training_log_ret=training_log_ret[training_log_ret.index.get_level_values('Date') >='2016/02/03']

In [57]:
alpha_nor

a51       a49       a12           a35       a54  \
ticker Date                                                               
AAPL   2016/02/03 -0.288753 -0.288753 -0.416899 -1.327336e-16 -0.865137   
       2016/02/04  0.020221  0.020221 -0.108073 -1.327336e-16 -0.394148   
       2016/02/05  0.559974  0.559974 -0.552250 -1.327336e-16  1.446707   
       2016/02/08 -0.120916 -0.120916 -0.249143 -1.327336e-16 -0.522728   
       2016/02/09  0.071718  0.071718 -0.064228 -1.327336e-16  0.053353   
...                     ...       ...       ...           ...       ...   
ZBRA   2020/12/23  0.110132  0.110132 -0.061921 -6.310006e-01  0.612775   
       2020/12/24  0.709318  0.709318 -0.660073 -6.865912e-01 -0.143807   
       2020/12/28 -1.238615 -1.238615 -1.308269  1.535110e-01 -1.106361   
       2020/12/29  1.096631  1.096631 -1.046718 -6.678628e-01  1.103220   
       2020/12/30 -1.240896 -1.240896  1.286776 -4.609587e-01 -0.571810   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2016/02/03 -1.285677  0.000000 -0.074604 -1.020275 -0.994222  ...   
       2016/02/04 -0.857112  0.000000  0.064509  0.140747  0.115370  ...   
       2016/02/05  0.692314  0.000000 -0.584489 -1.252479 -0.680186  ...   
       2016/02/08 -1.351610 -2.008652 -0.115930 -1.020275 -1.810714  ...   
       2016/02/09  0.197816  0.777291 -0.092648  2.172536  0.387534  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2020/12/23  0.265192  0.440153 -0.120103  0.166178 -0.399777  ...   
       2020/12/24  1.567567  0.702366  1.015162  1.748807  1.456013  ...   
       2020/12/28 -1.403476 -0.980344 -0.075342 -0.959923 -1.539297  ...   
       2020/12/29  1.526868  0.726980  0.515987  1.748807  0.479281  ...   
       2020/12/30 -1.322078 -0.089502 -0.539190 -0.868618 -1.457903  ...   

                         a5           a45       a23       a74       a13  \
ticker Date                                                               
AAPL   2016/02/03 -0.069897 -2.852414e-17  0.264835  1.087041  0.451078   
       2016/02/04 -0.082496 -2.852414e-17  0.264835  1.087041  0.516681   
       2016/02/05  0.867487 -2.852414e-17  0.264835  1.087041 -0.139350   
       2016/02/08  2.071135 -2.852414e-17  0.264835  1.087041 -0.139350   
       2016/02/09  1.211026 -1.991817e-01  0.264835  1.087041 -1.123395   
...                     ...           ...       ...       ...       ...   
ZBRA   2020/12/23 -0.257603 -1.002152e+00 -1.511068  0.803086  1.570189   
       2020/12/24 -0.788642 -1.028097e+00  1.425119  0.803086  1.418142   
       2020/12/28  0.850890 -1.469155e+00  0.421295  0.803086  1.076036   
       2020/12/29 -0.910822 -8.464846e-01 -0.429677  0.803086  1.000012   
       2020/12/30  1.359062  1.462586e+00 -0.117124  0.803086 -0.482448   

                            a84       a81       a11       a21       a14  
ticker Date                                                              
AAPL   2016/02/03  5.460353e-16  0.757490 -0.915863  0.629414 -0.213018  
       2016/02/04  5.460353e-16  0.757490  0.727112  0.629414 -0.274093  
       2016/02/05  5.460353e-16  0.757490  0.774188  0.629414 -0.210418  
       2016/02/08  5.460353e-16  0.757490  0.678557  0.629414 -0.705986  
       2016/02/09  5.460353e-16  0.757490 -0.862625  0.629414 -0.387017  
...                         ...       ...       ...       ...       ...  
ZBRA   2020/12/23 -1.417555e+00 -1.327087  0.483595  0.553381  0.269060  
       2020/12/24 -7.869338e-01  0.753530  3.238359  0.553381 -0.034703  
       2020/12/28  9.247524e-01  0.753530 -0.367688  0.553381  1.191359  
       2020/12/29  9.247524e-01  0.753530  0.197822  0.553381 -0.016905  
       2020/12/30  1.014841e+00  0.753530 -1.412842  0.553381  1.672465  

[112567 rows x 30 columns]

In [58]:
alpha_test_2021.loc['ZBRA']

,a51,a49,a12,a35,a54,a25,a17,a38,a18,a34,...,a5,a45,a23,a74,a13,a84,a81,a11,a21,a14
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/03,1.500000,1.500000,-1.500000,704.0,-0.201671,0.538462,-0.364237,-0.306968,-0.153846,0.461538,...,0.185847,-0.114962,-11.810028,0,-0.510989,0.219780,0,0.780099,-1,0.468840
2021/02/04,-3.649994,-3.649994,3.649994,704.0,-0.642979,0.329670,-0.364237,-0.229139,-0.714286,0.587912,...,0.546552,-0.114962,0.000000,0,-0.450549,0.219780,0,0.578916,-1,0.076665
2021/02/05,0.230011,0.230011,0.230011,704.0,-0.246496,0.648352,-0.364237,-0.254317,-0.527473,0.769231,...,0.126796,-0.114962,0.250000,0,-0.406593,0.219780,0,0.810289,-1,0.114349
2021/02/08,-13.200012,-13.200012,13.200012,704.0,-0.904538,0.670330,-0.460259,-0.720263,-0.956044,0.027473,...,0.787948,-0.114962,-10.950012,0,-0.472527,0.219780,0,0.560319,-1,0.567022
2021/02/09,-4.959991,-4.959991,-4.959991,704.0,-0.917630,0.373626,-0.089587,-0.706436,-0.901099,0.087912,...,0.841686,-0.483033,-13.509979,0,-0.461538,0.219780,0,0.641589,-1,0.184809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/12/23,-1.979980,-1.979980,-1.979980,1302.0,-0.459641,0.307692,-0.042040,-0.016906,-0.285714,0.675824,...,0.004830,-0.743872,0.000000,0,-0.054945,0.087912,0,1.470837,1,0.524067
2021/12/27,-23.920044,-23.920044,-23.920044,14880.0,-0.809768,0.197802,-0.627943,-0.058447,-0.978022,0.043956,...,0.054945,-0.763193,-24.800049,0,-0.434066,0.703297,-1,0.792296,1,0.456526
2021/12/28,9.010010,9.010010,-9.010010,160.0,-0.229235,0.835165,-0.000920,-0.123657,-0.241758,0.516484,...,0.056515,-0.096607,-22.460022,0,-0.626374,0.626374,0,0.567323,-1,0.115708


In [59]:
x=alpha_nor.loc['ZBRA']
x_test_2021 = alpha_test_2021.loc[[ticker]]

y=training_log_ret['ZBRA']
clf=linear_model.Lasso(alpha=0.001)
clf.fit(x, y)
clf.coef_

array([-1.94062331e-02, -1.79502510e-19, -0.00000000e+00,  2.09551437e-04,
       -0.00000000e+00, -0.00000000e+00, -5.23824274e-04, -0.00000000e+00,
       -3.73726491e-04, -0.00000000e+00, -0.00000000e+00, -1.63509693e-03,
       -0.00000000e+00,  4.94803001e-04,  0.00000000e+00,  0.00000000e+00,
       -3.40395591e-03, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [60]:
alpha_test_2021

a51        a49        a12           a35       a54  \
ticker Date                                                                  
AAPL   2021/02/03   1.050003   1.050003   1.050003   2490.237443 -0.163444   
       2021/02/04  -3.449997  -3.449997   3.449997   2490.237443 -0.957539   
       2021/02/05   0.630005   0.630005  -0.630005   2490.237443 -0.589473   
       2021/02/08  -0.150009  -0.150009   0.150009   2490.237443 -0.944539   
       2021/02/09   0.900009   0.900009   0.900009   2490.237443 -0.080595   
...                      ...        ...        ...           ...       ...   
ZBRA   2021/12/23  -1.979980  -1.979980  -1.979980   1302.000000 -0.459641   
       2021/12/27 -23.920044 -23.920044 -23.920044  14880.000000 -0.809768   
       2021/12/28   9.010010   9.010010  -9.010010    160.000000 -0.229235   
       2021/12/29  -3.799988  -3.799988   3.799988    294.000000 -0.639683   
       2021/12/30   3.580017   3.580017  -3.580017     78.000000 -0.154421   

                        a25       a17       a38       a18       a34  ...  \
ticker Date                                                          ...   
AAPL   2021/02/03  0.802198 -0.137292 -0.084169 -0.527473  0.901099  ...   
       2021/02/04  0.373626 -0.137292 -0.197802 -0.538462  0.412088  ...   
       2021/02/05  0.890110 -0.137292 -0.101497 -0.406593  0.219780  ...   
       2021/02/08  0.472527 -0.018434 -0.121724 -0.538462  0.747253  ...   
       2021/02/09  0.956044 -0.051722 -0.099746 -0.175824  0.857143  ...   
...                     ...       ...       ...       ...       ...  ...   
ZBRA   2021/12/23  0.307692 -0.042040 -0.016906 -0.285714  0.675824  ...   
       2021/12/27  0.197802 -0.627943 -0.058447 -0.978022  0.043956  ...   
       2021/12/28  0.835165 -0.000920 -0.123657 -0.241758  0.516484  ...   
       2021/12/29  0.186813 -0.076886 -0.266997 -0.967033  0.175824  ...   
       2021/12/30  0.780220 -0.004802 -0.079882 -0.791209  0.956044  ...   

                         a5       a45        a23  a74       a13       a84  \
ticker Date                                                                 
AAPL   2021/02/03  0.051926 -0.000218  -0.389999    0 -0.813187  0.517015   
       2021/02/04  0.147084 -0.000218  -1.089996    0 -0.813187  0.517015   
       2021/02/05  0.085135 -0.000218  -1.649994    0 -0.780220  0.517015   
       2021/02/08  0.104818 -0.000218   0.439987    0 -0.725275  0.517015   
       2021/02/09  0.040575  0.294590  -0.460007    0 -0.725275  0.517015   
...                     ...       ...        ...  ...       ...       ...   
ZBRA   2021/12/23  0.004830 -0.743872   0.000000    0 -0.054945  0.087912   
       2021/12/27  0.054945 -0.763193 -24.800049    0 -0.434066  0.703297   
       2021/12/28  0.056515 -0.096607 -22.460022    0 -0.626374  0.626374   
       2021/12/29  0.639053  0.859585   3.750000    0 -0.450549  0.835165   
       2021/12/30  0.073421 -0.696413   3.830017    0 -0.626374  0.846154   

                   a81       a11  a21       a14  
ticker Date                                      
AAPL   2021/02/03    0  0.013404    1 -0.224561  
       2021/02/04    0  0.023910    1 -0.264246  
       2021/02/05    0  0.070161    1 -0.173221  
       2021/02/08    0  0.030914    1 -0.094295  
       2021/02/09    0  0.082840    1 -0.091278  
...                ...       ...  ...       ...  
ZBRA   2021/12/23    0  1.470837    1  0.524067  
       2021/12/27   -1  0.792296    1  0.456526  
       2021/12/28    0  0.567323   -1  0.115708  
       2021/12/29    0  0.405024    1  0.584279  
       2021/12/30    0  0.331361    1  0.015954  

[20930 rows x 30 columns]

In [61]:
alpha5_result=pd.DataFrame(columns=alpha.columns)
alpha5_penalty_result=pd.DataFrame(columns=['penalty'])

backtest_y_2021=pd.DataFrame(columns=tickers, index=alpha_test_2021.loc['AAPL'].index)
backtest_y_2021_2022=pd.DataFrame(columns=tickers, index=alpha_test_2021_2022.loc['AAPL'].index)

penalty=np.logspace(np.log10(0.0001), np.log10(0.05), 300)

for ticker in tickers:
    for i in penalty:
        x=alpha_nor.loc[[ticker]]
        x_test_2021 = alpha_test_nor_2021.loc[[ticker]]
        x_test_2021_2022 = alpha_test_nor_2021_2022.loc[[ticker]]
        y=training_log_ret[ticker]
        clf=linear_model.Lasso(alpha=i)
        clf.fit(x, y)
        n=np.count_nonzero(clf.coef_)
        if n<=7 and n>=3:
            alpha5_result.loc[ticker]=clf.coef_
            alpha5_penalty_result.loc[ticker] = i
            y_predict_2021 = clf.predict(x_test_2021)
            backtest_y_2021[ticker]=y_predict_2021
            
            y_predict_2021_2022 = clf.predict(x_test_2021_2022)
            backtest_y_2021_2022[ticker]=y_predict_2021_2022
            break

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.916e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.920e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.958e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.959e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.984e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.987e-05, tolerance: 4.892e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

In [62]:
backtest_y_2021

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/03,-0.009665,0.014629,-0.021671,0.059380,0.061611,-0.031499,-0.002020,0.003426,0.012181,-0.018364,...,-0.003483,-0.003732,-0.000602,-0.006929,0.005141,0.014208,-0.001734,-0.002653,0.007945,-0.006413
2021/02/04,0.024293,-0.006774,0.009035,-0.004345,-0.005103,-0.010466,-0.002056,0.015345,0.055731,0.027559,...,0.011046,0.030158,-0.002971,0.035108,0.022677,0.022115,0.000429,-0.009658,0.035882,0.028240
2021/02/05,-0.005578,0.001259,0.009958,0.014958,0.015483,0.001113,0.005920,0.000525,0.000919,-0.004211,...,-0.000672,0.060741,0.014216,0.007326,0.022298,-0.003821,-0.010487,-0.000741,0.013576,0.008427
2021/02/08,0.003036,0.001604,-0.010762,-0.004084,-0.004120,0.001468,-0.006375,0.038422,0.040210,-0.007815,...,0.037305,0.016065,0.001869,0.018315,0.048914,0.022330,-0.006958,-0.007894,0.019481,0.040844
2021/02/09,-0.007894,0.006146,-0.007735,-0.006555,-0.006510,-0.023174,0.009821,-0.004039,0.009008,0.024894,...,0.021310,0.003478,0.060033,-0.012178,0.009827,-0.006333,0.005600,0.001409,-0.021761,-0.008439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/12/23,0.007904,0.006481,0.000854,0.003354,-0.000136,0.066261,0.015854,0.003243,0.003301,-0.002067,...,-0.005581,0.028028,0.003235,0.021132,0.015130,0.007871,0.014468,0.009408,0.012348,0.025363
2021/12/27,0.030401,0.031711,-0.011878,0.008021,0.007560,0.027975,0.035163,0.042767,-0.000086,-0.003972,...,0.056044,-0.003358,-0.002245,-0.008433,0.040279,-0.011206,0.001625,0.004107,0.010246,0.014411
2021/12/28,-0.009676,-0.007780,0.009037,-0.012575,-0.016588,-0.011320,0.000812,-0.024198,-0.005848,-0.006276,...,-0.021311,-0.007999,0.000095,-0.022707,-0.019378,-0.011070,-0.003043,-0.007083,0.005369,0.012203


In [63]:
backtest_y_2021_2022

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/03,-0.006484,0.013042,-0.015982,0.047148,0.048116,-0.021262,0.000092,-0.000172,0.013764,-0.013620,...,-0.000548,-0.005011,-0.000331,-0.006544,0.004842,0.015331,-0.000982,-0.003723,0.009592,-0.005704
2021/02/04,0.020038,-0.003443,0.008920,-0.001653,-0.002109,-0.001090,-0.000108,0.011272,0.064221,0.023482,...,0.013316,0.025053,-0.002774,0.030080,0.020205,0.022941,0.000817,-0.010771,0.039873,0.032068
2021/02/05,-0.003083,0.003203,0.009641,0.013434,0.013680,0.010102,0.006791,-0.002875,0.000764,-0.002690,...,0.002162,0.052175,0.015239,0.005923,0.019917,-0.001693,-0.009149,-0.001819,0.015634,0.010201
2021/02/08,0.004154,0.003456,-0.007656,-0.001490,-0.001402,0.010011,-0.003660,0.032236,0.045815,-0.005568,...,0.038079,0.012551,0.002275,0.015690,0.043298,0.023109,-0.005885,-0.008995,0.022035,0.046525
2021/02/09,-0.005046,0.006894,-0.005407,-0.003457,-0.003297,-0.013229,0.010000,-0.007174,0.010470,0.021375,...,0.022262,0.002760,0.063456,-0.010815,0.009067,-0.004105,0.005757,0.000348,-0.022415,-0.006683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022/12/23,-0.002754,0.004691,0.010537,0.011732,0.012203,-0.019832,0.005040,-0.006433,0.005769,-0.005096,...,0.006630,-0.018594,-0.007362,-0.020719,-0.002380,0.012746,-0.002525,0.005852,0.000818,-0.003175
2022/12/27,-0.008802,-0.006029,-0.011523,-0.012195,-0.012377,-0.046083,-0.002243,-0.039923,-0.000420,-0.017406,...,0.013089,0.033694,0.018135,-0.033638,0.023428,0.000371,-0.000712,-0.010178,-0.000684,0.010271
2022/12/28,-0.020516,-0.008752,-0.006332,-0.009365,-0.009914,0.006879,-0.002633,-0.005731,-0.002304,-0.012894,...,-0.007783,-0.023748,-0.008655,-0.006903,-0.002264,-0.015411,-0.007840,-0.008668,-0.001775,-0.022386


In [64]:
alpha5_result

,a51,a49,a12,a35,a54,a25,a17,a38,a18,a34,...,a5,a45,a23,a74,a13,a84,a81,a11,a21,a14
AAPL,-0.013493,-0.000000e+00,0.000000,0.001734,-0.001006,-0.002494,-0.0,-0.000000,0.000870,0.0,...,-0.000000,-0.000000,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.000000
MSFT,-0.013764,-0.000000e+00,-0.000000,0.001145,-0.000656,-0.001677,-0.0,0.000000,0.000000,-0.0,...,-0.000000,-0.000000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.000000
AMZN,-0.013395,-0.000000e+00,-0.000000,0.001202,-0.000636,-0.002225,-0.0,-0.000000,0.000000,-0.0,...,-0.000000,-0.000000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.000000
GOOGL,-0.015305,-0.000000e+00,0.000000,0.000162,-0.000374,-0.000670,-0.0,0.000000,0.000043,-0.0,...,-0.000000,-0.000000,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.000000
GOOG,-0.015312,-0.000000e+00,0.000000,0.000107,-0.000577,-0.000442,-0.0,-0.000000,0.000000,-0.0,...,-0.000000,-0.000000,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXPE,-0.000000,-2.711270e-02,0.001907,-0.000000,0.000283,0.000822,0.0,-0.000000,0.000000,0.0,...,-0.000000,-0.000000,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.000113
CSGP,-0.014414,-0.000000e+00,-0.000000,0.001754,-0.001672,0.000478,-0.0,-0.000149,-0.000000,-0.0,...,-0.000000,0.000000,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000
SIRI,-0.017110,-4.936319e-19,0.000000,0.000000,-0.000000,0.000052,-0.0,-0.000000,-0.000000,0.0,...,-0.000000,-0.000128,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.000018
EBAY,-0.016370,-2.198906e-18,-0.000000,0.000727,-0.000000,0.000000,-0.0,-0.000000,0.000000,0.0,...,-0.000048,-0.000000,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.000000


In [65]:
backtest_y_2021_2022.isna().sum()

AAPL     0
MSFT     0
AMZN     0
GOOGL    0
GOOG     0
        ..
EXPE     0
CSGP     0
SIRI     0
EBAY     0
WDC      0
Length: 91, dtype: int64

In [66]:
alpha5_result.to_csv('lasso_alpha.csv')

In [67]:
backtest_y_2021.to_csv('backtest y 2021.csv')

In [68]:
backtest_y_2021_2022.to_csv('backtest y 2021&2022.csv')

In [69]:
(np.exp(backtest_y_2021)-1).mean()*252

AAPL     0.406796
MSFT     0.344806
AMZN     0.400094
GOOGL    0.195908
GOOG     0.200347
           ...   
EXPE     0.149520
CSGP     0.381517
SIRI     0.163695
EBAY     0.201214
WDC      0.153495
Length: 91, dtype: float64

In [70]:
(np.exp(backtest_y_2021_2022)-1).mean()*252

AAPL     0.406587
MSFT     0.344804
AMZN     0.399508
GOOGL    0.196050
GOOG     0.200494
           ...   
EXPE     0.150561
CSGP     0.382872
SIRI     0.163662
EBAY     0.201289
WDC      0.158018
Length: 91, dtype: float64

In [71]:
backtest_y_2021

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/03,-0.009665,0.014629,-0.021671,0.059380,0.061611,-0.031499,-0.002020,0.003426,0.012181,-0.018364,...,-0.003483,-0.003732,-0.000602,-0.006929,0.005141,0.014208,-0.001734,-0.002653,0.007945,-0.006413
2021/02/04,0.024293,-0.006774,0.009035,-0.004345,-0.005103,-0.010466,-0.002056,0.015345,0.055731,0.027559,...,0.011046,0.030158,-0.002971,0.035108,0.022677,0.022115,0.000429,-0.009658,0.035882,0.028240
2021/02/05,-0.005578,0.001259,0.009958,0.014958,0.015483,0.001113,0.005920,0.000525,0.000919,-0.004211,...,-0.000672,0.060741,0.014216,0.007326,0.022298,-0.003821,-0.010487,-0.000741,0.013576,0.008427
2021/02/08,0.003036,0.001604,-0.010762,-0.004084,-0.004120,0.001468,-0.006375,0.038422,0.040210,-0.007815,...,0.037305,0.016065,0.001869,0.018315,0.048914,0.022330,-0.006958,-0.007894,0.019481,0.040844
2021/02/09,-0.007894,0.006146,-0.007735,-0.006555,-0.006510,-0.023174,0.009821,-0.004039,0.009008,0.024894,...,0.021310,0.003478,0.060033,-0.012178,0.009827,-0.006333,0.005600,0.001409,-0.021761,-0.008439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/12/23,0.007904,0.006481,0.000854,0.003354,-0.000136,0.066261,0.015854,0.003243,0.003301,-0.002067,...,-0.005581,0.028028,0.003235,0.021132,0.015130,0.007871,0.014468,0.009408,0.012348,0.025363
2021/12/27,0.030401,0.031711,-0.011878,0.008021,0.007560,0.027975,0.035163,0.042767,-0.000086,-0.003972,...,0.056044,-0.003358,-0.002245,-0.008433,0.040279,-0.011206,0.001625,0.004107,0.010246,0.014411
2021/12/28,-0.009676,-0.007780,0.009037,-0.012575,-0.016588,-0.011320,0.000812,-0.024198,-0.005848,-0.006276,...,-0.021311,-0.007999,0.000095,-0.022707,-0.019378,-0.011070,-0.003043,-0.007083,0.005369,0.012203


In [74]:
na=pd.DataFrame(index=[i for i in range(19)], columns=backtest_y_2021.columns)
backtest_y_2021=pd.concat([na, backtest_y_2021])

In [78]:
na

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
pd.concat([na, backtest_y_2021])

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/12/23,0.007904,0.006481,0.000854,0.003354,-0.000136,0.066261,0.015854,0.003243,0.003301,-0.002067,...,-0.005581,0.028028,0.003235,0.021132,0.015130,0.007871,0.014468,0.009408,0.012348,0.025363
2021/12/27,0.030401,0.031711,-0.011878,0.008021,0.007560,0.027975,0.035163,0.042767,-0.000086,-0.003972,...,0.056044,-0.003358,-0.002245,-0.008433,0.040279,-0.011206,0.001625,0.004107,0.010246,0.014411
2021/12/28,-0.009676,-0.007780,0.009037,-0.012575,-0.016588,-0.011320,0.000812,-0.024198,-0.005848,-0.006276,...,-0.021311,-0.007999,0.000095,-0.022707,-0.019378,-0.011070,-0.003043,-0.007083,0.005369,0.012203
2021/12/29,0.002378,0.003538,-0.010836,-0.002001,-0.000444,-0.001881,-0.012070,-0.011517,0.000972,-0.001863,...,0.011775,-0.009974,-0.012798,0.003980,0.021970,-0.000327,0.002908,-0.006973,0.004821,0.054842
